In [2]:
#import libraries
import pandas as pd 
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz

In [3]:
#import data
%store -r train_data
%store -r test_data
%store -r train_target
%store -r test_target

In [5]:
tree_clf = DecisionTreeClassifier(max_depth = 20)

tree_clf.fit(train_data, train_target)

DecisionTreeClassifier(max_depth=20)

In [8]:
from sklearn import tree

text_representation_tree_1 = tree.export_text(tree_clf)
print(text_representation_tree_1)


|--- feature_16 <= 6.50
|   |--- feature_3 <= 49.50
|   |   |--- feature_19 <= 0.50
|   |   |   |--- feature_7 <= 124.50
|   |   |   |   |--- feature_3 <= 5.50
|   |   |   |   |   |--- feature_3 <= 4.50
|   |   |   |   |   |   |--- feature_0 <= 2.50
|   |   |   |   |   |   |   |--- class: 2
|   |   |   |   |   |   |--- feature_0 >  2.50
|   |   |   |   |   |   |   |--- feature_11 <= 23.00
|   |   |   |   |   |   |   |   |--- class: 4
|   |   |   |   |   |   |   |--- feature_11 >  23.00
|   |   |   |   |   |   |   |   |--- feature_7 <= 121.50
|   |   |   |   |   |   |   |   |   |--- feature_2 <= 12.50
|   |   |   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |   |   |   |--- feature_2 >  12.50
|   |   |   |   |   |   |   |   |   |   |--- class: 2
|   |   |   |   |   |   |   |   |--- feature_7 >  121.50
|   |   |   |   |   |   |   |   |   |--- class: 3
|   |   |   |   |   |--- feature_3 >  4.50
|   |   |   |   |   |   |--- feature_0 <= 1.50
|   |   |   |   |   |   |  

In [14]:
from matplotlib import pyplot as plt

tree.plot_tree(tree_clf)

[Text(178.60342663242605, 212.26285714285714, 'X[16] <= 6.5\ngini = 0.835\nsamples = 423680\nvalue = [56617, 78010, 103542, 21915, 14277, 38099, 84615, 7294\n1025, 18250, 10, 22, 4]'),
 Text(90.68953966607033, 201.90857142857143, 'X[3] <= 49.5\ngini = 0.836\nsamples = 238198\nvalue = [36918, 24493, 22556, 10586, 12323, 34075, 70913, 7183\n954, 18162, 9, 22, 4]'),
 Text(27.068299697471613, 191.5542857142857, 'X[19] <= 0.5\ngini = 0.798\nsamples = 187733\nvalue = [34076, 23563, 21709, 8455, 5734, 14196, 67487, 5164\n484, 6850, 4, 9, 2]'),
 Text(5.025236744367995, 181.2, 'X[7] <= 124.5\ngini = 0.572\nsamples = 9549\nvalue = [179, 854, 6024, 773, 59, 367, 1086, 152, 12, 41, 0\n2, 0]'),
 Text(1.1497563522835441, 170.84571428571428, 'X[3] <= 5.5\ngini = 0.431\nsamples = 4347\nvalue = [30, 292, 3216, 538, 5, 62, 171, 23, 6, 4, 0, 0\n0]'),
 Text(0.16965671197018625, 160.49142857142857, 'X[3] <= 4.5\ngini = 0.607\nsamples = 832\nvalue = [10, 118, 445, 244, 0, 1, 12, 2, 0, 0, 0, 0, 0]'),
 Text(0

Error in callback <function flush_figures at 0x7faa88bd6430> (for post_execute):


KeyboardInterrupt: 

In [13]:
#print(train_data)

[[ 5  3  6 ... 14 -1  7]
 [16  9  0 ... 14 -1  7]
 [ 2  7 18 ... 14 -1  7]
 ...
 [ 1  7 14 ... 22 -1  7]
 [ 1  3  7 ...  7 -1  4]
 [ 3  3 22 ... 14 -1  7]]


Now starting the random forests.

In [15]:
from sklearn.ensemble import RandomForestClassifier

In [22]:
rnd_clf = RandomForestClassifier(n_estimators = 500, max_leaf_nodes = 16, n_jobs = 1, random_state = 42)
rnd_clf.fit(train_data, train_target)

y_pred_rf = rnd_clf.predict(test_data)

In [19]:
%store -r model_datalist

In [20]:
print(model_datalist)

['exp_years', 'date_stop', 'time_stop', 'stopduration', 'stop_in_response_to_cfs', 'officer_assignment_key', 'isschool', 'beat', 'pid', 'isstudent', 'perceived_limited_english', 'perceived_age', 'gender_nonconforming', 'gend', 'lgbt_cat', 'race_cat', 'stop_cat', 'action_cat', 'consented_cat', 'contraband_cat']


In [23]:
#feature importance 

for name, score in zip(model_datalist,
rnd_clf.feature_importances_):
    print(name, score)

exp_years 0.13807264375961922
date_stop 0.00019261985471422604
time_stop 0.0120583074385014
stopduration 0.22217050925345475
stop_in_response_to_cfs 0.02803030179642627
officer_assignment_key 0.011412756189462607
isschool 0.0
beat 0.028225477315889272
pid 0.03662637169321379
isstudent 0.0
perceived_limited_english 0.0
perceived_age 0.003383713080294506
gender_nonconforming 0.0
gend 0.0017445203946191586
lgbt_cat 2.798870128696345e-05
race_cat 0.0020881722101376444
stop_cat 0.36212971925146237
action_cat 0.10329055398132371
consented_cat 0.0003554244595903483
contraband_cat 0.050190920620003826
